In [1]:
import pandas as pd
from cassandra.cluster import Cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

In [3]:
session.execute("DROP KEYSPACE IF EXISTS m14")

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':2
    }
""")

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    SKU TEXT, 
    prod_name TEXT, 
    description TEXT, 
    warehouse_num INT, 
    PRIMARY KEY(SKU, warehouse_num)
);
""")

In [8]:
df = pd.read_csv('data.csv')
df

,SKU,prod_name,description,warehouse_num
0,'AB1234','Laptop','MACBook Air',1
1,'AB7854','Chair','Office chairs that are adjustable',3
2,'DW6278','Printer','HP Smart Tank 5101 All-in-One Printer',1
3,'GT9837','Cable','Electrical cables',2
4,'FO9862','Table','Office tables-foldable',3
5,'DW3728','Sofa','An L shape leather sofa set',2
6,'SO5569','Bag','Converse Laptop Bag',1
7,'SO8886','Hanger','Clothes hangers set of 12',3
8,'WE9226','Mirror','Over the door mirror',2
9,'EW6671','Phone','Iphone 14 pro max',3


In [9]:
for index, row in df.iterrows():
    print(f"SKU = {row.SKU}, prod_name ={row.prod_name}, description = {row.description}, warehouse_num = {row.warehouse_num}")

SKU = 'AB1234', prod_name ='Laptop', description = 'MACBook Air', warehouse_num = 1
SKU = 'AB7854', prod_name ='Chair', description = 'Office chairs that are adjustable', warehouse_num = 3
SKU = 'DW6278', prod_name ='Printer', description = 'HP Smart Tank 5101 All-in-One Printer', warehouse_num = 1
SKU = 'GT9837', prod_name ='Cable', description = 'Electrical cables', warehouse_num = 2
SKU = 'FO9862', prod_name ='Table', description = 'Office tables-foldable', warehouse_num = 3
SKU = 'DW3728', prod_name ='Sofa', description = 'An L shape leather sofa set', warehouse_num = 2
SKU = 'SO5569', prod_name ='Bag', description = 'Converse Laptop Bag', warehouse_num = 1
SKU = 'SO8886', prod_name ='Hanger', description = 'Clothes hangers set of 12', warehouse_num = 3
SKU = 'WE9226', prod_name ='Mirror', description = 'Over the door mirror', warehouse_num = 2
SKU = 'EW6671', prod_name ='Phone', description = 'Iphone 14 pro max', warehouse_num = 3


In [10]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (SKU, prod_name, description, warehouse_num)     
        VALUES ({row.SKU}, {row.prod_name}, {row.description}, {row.warehouse_num});
        """
    )
    session.execute(f"""
        INSERT INTO m14.inventory (SKU, prod_name, description, warehouse_num)     
        VALUES ({row.SKU}, {row.prod_name}, {row.description}, {row.warehouse_num});
        """
    )


        INSERT INTO m14.inventory (SKU, prod_name, description, warehouse_num)     
        VALUES ('AB1234', 'Laptop', 'MACBook Air', 1);
        

        INSERT INTO m14.inventory (SKU, prod_name, description, warehouse_num)     
        VALUES ('AB7854', 'Chair', 'Office chairs that are adjustable', 3);
        

        INSERT INTO m14.inventory (SKU, prod_name, description, warehouse_num)     
        VALUES ('DW6278', 'Printer', 'HP Smart Tank 5101 All-in-One Printer', 1);
        

        INSERT INTO m14.inventory (SKU, prod_name, description, warehouse_num)     
        VALUES ('GT9837', 'Cable', 'Electrical cables', 2);
        

        INSERT INTO m14.inventory (SKU, prod_name, description, warehouse_num)     
        VALUES ('FO9862', 'Table', 'Office tables-foldable', 3);
        

        INSERT INTO m14.inventory (SKU, prod_name, description, warehouse_num)     
        VALUES ('DW3728', 'Sofa', 'An L shape leather sofa set', 2);
        

        INSERT INTO m14.inve

In [11]:
rows = session.execute("select (SKU, prod_name, description, warehouse_num) from m14.inventory")
for row in rows:
    print(f"SKU={row[0][0]}, prod_name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


SKU=GT9837, prod_name=Cable, description=Electrical cables, warehouse_num=2
SKU=FO9862, prod_name=Table, description=Office tables-foldable, warehouse_num=3
SKU=DW3728, prod_name=Sofa, description=An L shape leather sofa set, warehouse_num=2
SKU=SO5569, prod_name=Bag, description=Converse Laptop Bag, warehouse_num=1
SKU=SO8886, prod_name=Hanger, description=Clothes hangers set of 12, warehouse_num=3
SKU=WE9226, prod_name=Mirror, description=Over the door mirror, warehouse_num=2
SKU=DW6278, prod_name=Printer, description=HP Smart Tank 5101 All-in-One Printer, warehouse_num=1
SKU=AB1234, prod_name=Laptop, description=MACBook Air, warehouse_num=1
SKU=EW6671, prod_name=Phone, description=Iphone 14 pro max, warehouse_num=3
SKU=AB7854, prod_name=Chair, description=Office chairs that are adjustable, warehouse_num=3


In [12]:
# Create an index on warehouse_num in the inventory table
session.execute("""
CREATE INDEX IF NOT EXISTS warehouse_num
ON m14.inventory (warehouse_num);
""")

In [13]:
rows = session.execute("SELECT (sku, prod_name, description, warehouse_num) FROM m14.inventory WHERE warehouse_num = 2 ")
for row in rows:
    print(f"{row[0][0]}, {row[0][1]}, {row[0][2]}, {row[0][3]}")

GT9837, Cable, Electrical cables, 2
DW3728, Sofa, An L shape leather sofa set, 2
WE9226, Mirror, Over the door mirror, 2
